In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [2]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

2017-06-18 13:42:57,341 [INFO] [holmes.experiment.staging] Preparing holmes payload
2017-06-18 13:42:57,342 [INFO] [holmes.experiment.staging] Cloning git:hudl/holmes
2017-06-18 13:42:59,964 [INFO] [holmes.experiment.staging] Checking out git:hudl/holmes:ca6ccd63f270d603ffa9bdf5a7ced184d6339796
2017-06-18 13:43:00,328 [INFO] [holmes.experiment.staging] Preparing distribution package
2017-06-18 13:43:03,085 [INFO] [holmes.experiment.staging] Staging distribution package
2017-06-18 13:43:03,088 [INFO] [holmes.experiment.staging] Commit staged
2017-06-18 13:43:03,089 [INFO] [holmes.experiment.staging] Removing Holmes Directory
2017-06-18 13:43:03,154 [INFO] [holmes.experiment.staging] Extracting default config from package


In [3]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [1]
hungarian_thresholds = [10,15]
minimum_detections = [5,10]

In [4]:
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

16 /var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmptsyh7qw4/M00-ChvUtd-000-01-10-05.json


In [5]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-2-2", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-2-2",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

2017-06-18 13:43:03,267 [INFO] [holmes.experiment.base] Verifying AWS staging area: s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2


In [6]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [7]:
experiment.stage_experiment()

2017-06-18 13:43:05,847 [INFO] [holmes.experiment.base] staging measurement M00-ChvUtd-000-01-10-05
2017-06-18 13:43:06,320 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmptsyh7qw4/M00-ChvUtd-000-01-10-05.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/M00-ChvUtd-000-01-10-05.json
2017-06-18 13:43:06,940 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmptsyh7qw4/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz
2017-06-18 13:43:08,250 [INFO] [holmes.experiment.base] staging measurement M01-PS0304-001-01-10-05
2017-06-18 13:43:08,759 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmptsyh7qw4/M01-PS0304-001-01-10-05.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps

{'experiment_id': 'PTA-95-LT-HT-DET-2-2',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/M00-ChvUtd-000-01-10-05.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-2',
   'job_id': 'PTA-95-LT-HT-DET-2-2-M00-ChvUtd-000-01-10-05',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M00-ChvUtd-000-01-10-05',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/ca6ccd63f270d603ffa9bdf5a7ced184d6339796.tar.gz'},
  {'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/M01-PS0304-001-01-10-05.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-2',
   'job_id': 'PTA-95-LT-HT-DET-2-2-M01-PS0304-001-01-10-05',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M01-PS0304-001-01-10-05',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/ca6ccd63f270d603ffa9bdf5a7ced184d63397

In [8]:
experiment.launch_experiment()

2017-06-18 13:46:40,546 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M00-ChvUtd-000-01-10-05
2017-06-18 13:46:40,897 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M01-PS0304-001-01-10-05
2017-06-18 13:46:41,149 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M02-PP0214-000-01-10-05
2017-06-18 13:46:41,480 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M03-ER0429-001-01-10-05
2017-06-18 13:46:41,786 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M04-ChvUtd-000-01-10-10
2017-06-18 13:46:42,056 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M05-PS0304-001-01-10-10
2017-06-18 13:46:42,342 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M06-PP0214-000-01-10-10
2017-06-18 13:46:42,692 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M07-ER0429-001-01-10-10
2017-06-18 13:46:42,962 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-2-M08-ChvUtd-000-01-15-05
2

{'experiment_id': 'PTA-95-LT-HT-DET-2-2',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-2/deps/M00-ChvUtd-000-01-10-05.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-2',
   'job_id': 'PTA-95-LT-HT-DET-2-2-M00-ChvUtd-000-01-10-05',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '105',
      'content-type': 'application/json',
      'date': 'Sun, 18 Jun 2017 18:44:26 GMT',
      'via': '1.1 6abca978c7767ffcb2452cf2eadf7805.cloudfront.net (CloudFront)',
      'x-amz-cf-id': 'vdbmJZ9AEh7jOQymaiLS_oFApRv0jwk72oi1m4I91pRB-jBBNpeIfQ==',
      'x-amzn-requestid': '27498646-5456-11e7-a2fd-373f0589fce8',
      'x-amzn-trace-id': 'sampled=0;root=1-5946ca0a-3f7383d6ae4b275210a1191d',
      'x-cache': 'Miss from cloudfront'},
     'HTTPStatusCode': 200,
     'RequestId': '27498646-5456-11e7-a2fd-373f0589fce8',


In [10]:
import pandas as pd

In [11]:
data = pd.DataFrame({'mota':[.673,.679,.670,.677,.869,.871,.869,.874,.789,.787,.795,.798,.766,.766,.768,.771],
              'lt':[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
              'ht':[10,10,15,15,10,10,15,15,10,10,15,15,10,10,15,15],
              'det':[5,10,5,10,5,10,5,10,5,10,5,10,5,10,5,10],
             'game':['ER0429','ER0429','ER0429','ER0429','PP0214','PP0214','PP0214','PP0214','ChvUtd','ChvUtd',
                     'ChvUtd','ChvUtd','PS0304','PS0304','PS0304','PS0304']})

In [13]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [14]:
output_notebook()

Loading BokehJS ...

In [15]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections
ht lt det                                      
10 1  5    0.77425     1         10           5
      10   0.77575     1         10          10
15 1  5    0.77550     1         15           5
      10   0.78000     1         15          10

In [17]:
scat_lt_ht_det_1 = Scatter(data_lt_ht_det, x='hungarian', y='mota', color='detections', marker='lost')
show(scat_lt_ht_det_1)